# Testing DMD against rational adversaries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [7]:
time_horizon = 100
T = 6
#targets = util.gen_targets(T, low=1, high=10)
targets = util.gen_tar_with_len(T)
values = tuple((v, v) for v in targets)
print(values)
#obs = gen_observabilities_correlated_with_values(values)
#obs = {0: 0.75, 1: 0.75, 2: 0.75, 3: 0.75, 4: 0.75}
#obs_vect = [obs.get(i) for i in range(len(targets))]
#generating pure strat and some rand mixed strat stochastic attackers
distributions = []
for i in range(29):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)
chosen_distr = tuple(util.gen_distr(T))
print("chosen = ", chosen_distr)

((0.608, 0.608), (0.624, 0.624), (0.595, 0.595), (0.672, 0.672), (0.907, 0.907), (0.95, 0.95))
[(0.011, 0.15, 0.05, 0.313, 0.188, 0.288), (0.04395604395604396, 0.015984015984015987, 0.3056943056943057, 0.2457542457542458, 0.2317682317682318, 0.15684315684315686), (0.07207207207207206, 0.13413413413413414, 0.14014014014014015, 0.09709709709709709, 0.5235235235235235, 0.03303303303303303), (0.247, 0.228, 0.211, 0.275, 0.007, 0.032), (0.062062062062062065, 0.1091091091091091, 0.3433433433433434, 0.23423423423423426, 0.2172172172172172, 0.03403403403403404), (0.15015015015015015, 0.012012012012012012, 0.008008008008008008, 0.24524524524524524, 0.5635635635635635, 0.021021021021021023), (0.039, 0.102, 0.453, 0.362, 0.022, 0.022), (0.415, 0.224, 0.017, 0.142, 0.134, 0.068), (0.33266733266733267, 0.08791208791208789, 0.14585414585414583, 0.07492507492507491, 0.14985014985014983, 0.20879120879120877), (0.133, 0.746, 0.087, 0.029, 0.002, 0.003), (0.013, 0.078, 0.1, 0.014, 0.23, 0.565), (0.00099

In [8]:
g = game.Game(values, time_horizon)
g.attackers = [1]
g.defenders = [0]
d = [dmd.DirichletMultinomialDefender(g, 0)]
prof = []
for distr in distributions:
    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
#for i in range(10):
#    prof.append(attackers.SUQR(g,1))
chosen_a_prof = (attackers.StackelbergAttacker(g, 1))
a = [deepcopy(chosen_a_prof)]
prof.append(chosen_a_prof)
shuffle(prof)
print("Real attacker is: ", chosen_a_prof)
g.set_players(d, a, prof)

Real attacker is:  sta1


In [9]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "\n" 

In [10]:
def print_row(targets, time_horizon, d, a, p):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p]) ])+"\n"

In [11]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile\n'

In [12]:
defender =["FB1"]

In [13]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [14]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Bayesian vs FB"
batch_name = "fb"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [15]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings))

In [16]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Profile.20,Profile.21,Profile.22,Profile.23,Profile.24,Profile.25,Profile.26,Profile.27,Profile.28,Profile.29
0,FB1_vs_sta1,100,0.608,0.624,0.595,0.672,0.907,0.95,FB1,sta1,...,sto1-0.000999000999000999-0.2017982017982018-0...,sto1-0.04395604395604396-0.015984015984015987-...,sto1-0.17300000000000001-0.20900000000000002-0...,sto1-0.5564435564435565-0.05194805194805194-0....,sto1-0.024024024024024024-0.07707707707707707-...,sto1-0.05-0.071-0.245-0.448-0.103-0.083,sto1-0.33266733266733267-0.08791208791208789-0...,sto1-0.013-0.078-0.1-0.014-0.23-0.565,sto1-0.039-0.102-0.453-0.362-0.022-0.022,sto1-0.20720720720720723-0.11811811811811812-0...


### Parsing the csv to generate a batch containing the configurations and running them

In [8]:
#os.mkdir("../NewExperiments/Exp2/belief_max_obs_vs_no_obs")

In [9]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [10]:
b.parse_batch()

In [11]:
batches = []
batches.append(b)

In [17]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

### Collecting results of the runs

In [18]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the two configurations "step-by-step"

In [12]:
gm = deepcopy(batches[0].configurations[0].game)

In [13]:
gm

<Game values:[[0.772, 0.772], [0.914, 0.914], [0.94, 0.94], [0.758, 0.758], [0.728, 0.728], [0.858, 0.858]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <DirichletMultinomialDefender id:0 resources:1>} time_horizon:100>

In [14]:
e = runner.Experiment(gm)

In [15]:
print(gm.players[0].profiles[0])

sto1-0.27127127127127126-0.3383383383383384-0.13613613613613615-0.13013013013013014-0.04804804804804805-0.07607607607607607


In [16]:
for i in range(50):
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

Approximate distribution is: [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
{0: [0, 0, 1, 0, 0, 0], 1: [0, 1, 0, 0, 0, 0]}
{1: [1], 0: [2]}
Approximate distribution is: [0.14285714285714285, 0.2857142857142857, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]
Comparing non zero distr [0.14285714285714285, 0.2857142857142857, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]  and [0.27127127127127126, 0.3383383383383384, 0.13613613613613615, 0.13013013013013014, 0.04804804804804805, 0.07607607607607607]
Profile sto1-0.27127127127127126-0.3383383383383384-0.13613613613613615-0.13013013013013014-0.04804804804804805-0.07607607607607607  with strat [0.27127127127127126, 0.3383383383383384, 0.13613613613613615, 0.13013013013013014, 0.04804804804804805, 0.07607607607607607]  has ent 0.1259817812915681
Comparing non zero distr [0.14285714285714285

Comparing non zero distr [0.07692307692307693, 0.3076923076923077, 0.38461538461538464, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]  and [0.252, 0.541, 0.016, 0.072, 0.088, 0.031]
Profile sto1-0.252-0.541-0.016-0.072-0.088-0.031  with strat [0.252, 0.541, 0.016, 0.072, 0.088, 0.031]  has ent 1.0226772334109389
Comparing non zero distr [0.07692307692307693, 0.3076923076923077, 0.38461538461538464, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]  and [0.19823746235253212, 6.330188337607165e-07, 0.20577053021532718, 0.19762229678202797, 0.1963105053995343, 0.20205857223174467]
Profile suqr1-12.686-0.222  with strat [0.19823746235253212, 6.330188337607165e-07, 0.20577053021532718, 0.19762229678202797, 0.1963105053995343, 0.20205857223174467]  has ent 3.9777691512722186
Comparing non zero distr [0.07692307692307693, 0.3076923076923077, 0.38461538461538464, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]  and [0.19941530269586555, 5.0124548458

Comparing non zero distr [0.05555555555555555, 0.3888888888888889, 0.3888888888888889, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555]  and [0.024024024024024024, 0.12812812812812813, 0.4724724724724724, 0.13013013013013014, 0.2172172172172172, 0.028028028028028028]
Profile sto1-0.024024024024024024-0.12812812812812813-0.4724724724724724-0.13013013013013014-0.2172172172172172-0.028028028028028028  with strat [0.024024024024024024, 0.12812812812812813, 0.4724724724724724, 0.13013013013013014, 0.2172172172172172, 0.028028028028028028]  has ent 0.31760453162566593
Comparing non zero distr [0.05555555555555555, 0.3888888888888889, 0.3888888888888889, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555]  and [0.287, 0.102, 0.014, 0.258, 0.091, 0.248]
Profile sto1-0.287-0.102-0.014-0.258-0.091-0.248  with strat [0.287, 0.102, 0.014, 0.258, 0.091, 0.248]  has ent 1.5261502451478948
Comparing non zero distr [0.05555555555555555, 0.3888888888888889, 0.3888888888888889, 

Profile suqr1-9.499-0.874  with strat [0.19281131022164785, 1.635568138676653e-05, 0.22330622671466266, 0.19046644616403507, 0.1855373191901409, 0.2078623420281268]  has ent 4.126733072740445
Comparing non zero distr [0.045454545454545456, 0.4090909090909091, 0.4090909090909091, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456]  and [0.191, 0.048, 0.197, 0.197, 0.273, 0.094]
Profile sto1-0.191-0.048-0.197-0.197-0.273-0.094  with strat [0.191, 0.048, 0.197, 0.197, 0.273, 0.094]  has ent 0.9290835529419783
Comparing non zero distr [0.045454545454545456, 0.4090909090909091, 0.4090909090909091, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456]  and [0.19187605735749572, 2.9548970617506425e-05, 0.22629187062283496, 0.1892561955155785, 0.18376203434534022, 0.20878429318813285]
Profile suqr1-8.918-0.982  with strat [0.19187605735749572, 2.9548970617506425e-05, 0.22629187062283496, 0.1892561955155785, 0.18376203434534022, 0.20878429318813285]  has ent 3.88007985

Profile suqr1-10.585-0.262  with strat [0.20990829585014076, 0.050763293478262805, 0.04105590512754217, 0.24868288600895366, 0.3657090526461045, 0.08388056688899616]  has ent 1.0059171998370273
Comparing non zero distr [1.0]  and [0.436]
Profile sto1-0.078-0.123-0.132-0.05-0.436-0.181  with strat [0.078, 0.123, 0.132, 0.05, 0.436, 0.181]  has ent 0.8301130356331028
Comparing non zero distr [1.0]  and [1]
Profile sta1  with strat [0, 0, 0, 0, 1, 0]  has ent 0.0
Comparing non zero distr [1.0]  and [0.088]
Profile sto1-0.252-0.541-0.016-0.072-0.088-0.031  with strat [0.252, 0.541, 0.016, 0.072, 0.088, 0.031]  has ent 2.430418464503931
Comparing non zero distr [1.0]  and [0.4043835577032712]
Profile suqr1-12.686-0.222  with strat [0.20704870033099115, 0.037286766318381306, 0.028843381875222465, 0.2540154794639024, 0.4043835577032712, 0.0684221143082313]  has ent 0.90539145116074
Comparing non zero distr [1.0]  and [0.32553945495880693]
Profile suqr1-8.299-0.073  with strat [0.2094295184397

...

Comparing non zero distr [1.0]  and [0.305233146891449]
Profile suqr1-7.811-0.677  with strat [0.20699234861176685, 0.07777894677664485, 0.06734498038911736, 0.2329900190763853, 0.305233146891449, 0.10966055825463662]  has ent 1.1866793783543215
Comparing non zero distr [1.0]  and [0.17117117117117117]
Profile sto1-0.07107107107107107-0.09009009009009009-0.13613613613613615-0.29429429429429427-0.17117117117117117-0.23723723723723722  with strat [0.07107107107107107, 0.09009009009009009, 0.13613613613613615, 0.29429429429429427, 0.17117117117117117, 0.23723723723723722]  has ent 1.7650912221458936
Comparing non zero distr [1.0]  and [0.33464284381763704]
Profile suqr1-9.499-0.874  with strat [0.20913214222015755, 0.06406048674542701, 0.0538377599201663, 0.24132355990956197, 0.33464284381763704, 0.09700320738704997]  has ent 1.0946914537441015
Comparing non zero distr [1.0]  and [0.273]
Profile sto1-0.191-0.048-0.197-0.197-0.273-0.094  with strat [0.191, 0.048, 0.197, 0.197, 0.273, 0.094

Comparing non zero distr [1.0]  and [0.1871871871871872]
Profile sto1-0.07607607607607607-0.47147147147147145-0.04704704704704705-0.14414414414414414-0.1871871871871872-0.07407407407407407  with strat [0.07607607607607607, 0.47147147147147145, 0.04704704704704705, 0.14414414414414414, 0.1871871871871872, 0.07407407407407407]  has ent 1.6756461617939669
Comparing non zero distr [1.0]  and [0.3935218207663132]
Profile suqr1-12.022-0.164  with strat [0.20824675119502137, 0.04075389405051679, 0.0319134199969195, 0.25293018199714545, 0.3935218207663132, 0.07263393199408369]  has ent 0.9326187596346619
Comparing non zero distr [1.0]  and [0.3657090526461045]
Profile suqr1-10.585-0.262  with strat [0.20990829585014076, 0.050763293478262805, 0.04105590512754217, 0.24868288600895366, 0.3657090526461045, 0.08388056688899616]  has ent 1.0059171998370273
Comparing non zero distr [1.0]  and [0.436]
Profile sto1-0.078-0.123-0.132-0.05-0.436-0.181  with strat [0.078, 0.123, 0.132, 0.05, 0.436, 0.181]

Profile suqr1-12.485-0.815  with strat [0.2079255337397945, 0.04187679865537562, 0.033034472056004904, 0.2521516573312941, 0.39140261857149694, 0.07360891964603393]  has ent 0.9380185337730035
Comparing non zero distr [1.0]  and [0.38538538538538536]
Profile sto1-0.002002002002002002-0.028028028028028024-0.10010010010010009-0.07307307307307306-0.38538538538538536-0.41141141141141135  with strat [0.002002002002002002, 0.028028028028028024, 0.10010010010010009, 0.07307307307307306, 0.38538538538538536, 0.41141141141141135]  has ent 0.9535114443607694
Comparing non zero distr [1.0]  and [0.4161362409611383]
Profile suqr1-13.642-0.54  with strat [0.2052878177308867, 0.03390929917185287, 0.02593038589441784, 0.2546888965469592, 0.4161362409611383, 0.06404735969474509]  has ent 0.8767425700276206
Comparing non zero distr [1.0]  and [0.01]
Profile sto1-0.451-0.212-0.068-0.112-0.01-0.147  with strat [0.451, 0.212, 0.068, 0.112, 0.01, 0.147]  has ent 4.605170185988092
I'll best respond to sta1


Profile sto1-0.252-0.541-0.016-0.072-0.088-0.031  with strat [0.252, 0.541, 0.016, 0.072, 0.088, 0.031]  has ent 2.430418464503931
Comparing non zero distr [1.0]  and [0.4043835577032712]
Profile suqr1-12.686-0.222  with strat [0.20704870033099115, 0.037286766318381306, 0.028843381875222465, 0.2540154794639024, 0.4043835577032712, 0.0684221143082313]  has ent 0.90539145116074
Comparing non zero distr [1.0]  and [0.32553945495880693]
Profile suqr1-8.299-0.073  with strat [0.20942951843977423, 0.06753526811448006, 0.056985943854858376, 0.23964075818302574, 0.32553945495880693, 0.10086905644905465]  has ent 1.1222716112881508
Comparing non zero distr [1.0]  and [0.305233146891449]
Profile suqr1-7.811-0.677  with strat [0.20699234861176685, 0.07777894677664485, 0.06734498038911736, 0.2329900190763853, 0.305233146891449, 0.10966055825463662]  has ent 1.1866793783543215
Comparing non zero distr [1.0]  and [0.17117117117117117]
Profile sto1-0.07107107107107107-0.09009009009009009-0.1361361361

Profile sto1-0.024024024024024024-0.12812812812812813-0.4724724724724724-0.13013013013013014-0.2172172172172172-0.028028028028028028  with strat [0.024024024024024024, 0.12812812812812813, 0.4724724724724724, 0.13013013013013014, 0.2172172172172172, 0.028028028028028028]  has ent 1.5268574251080942
Comparing non zero distr [1.0]  and [0.091]
Profile sto1-0.287-0.102-0.014-0.258-0.091-0.248  with strat [0.287, 0.102, 0.014, 0.258, 0.091, 0.248]  has ent 2.396895772465287
Comparing non zero distr [1.0]  and [0.304359949387834]
Profile suqr1-7.565-0.457  with strat [0.2071218400722431, 0.07803134049690706, 0.06752102064548365, 0.23291563947936222, 0.304359949387834, 0.11005020991816994]  has ent 1.189544233964294
Comparing non zero distr [1.0]  and [0.1871871871871872]
Profile sto1-0.07607607607607607-0.47147147147147145-0.04704704704704705-0.14414414414414414-0.1871871871871872-0.07407407407407407  with strat [0.07607607607607607, 0.47147147147147145, 0.04704704704704705, 0.1441441441441

In [19]:
e.game

<Game values:[[0.902, 0.902], [0.9790000000000001, 0.9790000000000001], [0.848, 0.848], [0.976, 0.976], [0.956, 0.956], [0.912, 0.912]] players{1: <StochasticAttacker id:1 resources:1>, 0: <DirichletMultinomialDefender id:0 resources:1>} time_horizon:100>

In [28]:
e.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340925,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.190303301163861,
 <StochasticAttacker id:1 resources:1>: -1.2740214327993638,
 <StochasticAttacker id:1 resources:1>: -0.09126790561131946,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -0.1133238910766163,
 <StochasticAttacker id:1 resources:1>: -3.32124264636016,
 <StochasticAttacker id:1 resources:1>: -1.6956695963863746,
 <Stochasti

In [29]:
gm1 = deepcopy(batches[0].configurations[1].game)

In [30]:
gm1

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [31]:
e1 = runner.Experiment(gm1)

In [32]:
for i in range(900):
    e1.run_interaction_with_observations()
    print(e1.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e1.game.history[-1])
    print(e1.game.observation_history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: (0, 0, 0, 1, 0), 1: [0, 0, 0, 0, 1]}
{1: [4], 0: [3]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: (0, 0, 0, 0, 1), 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}


{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 